In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import re
import string
from fuzzywuzzy import fuzz
from merge_charlotte import parse_address, fuzz_comparisons, append_fuzz_scores
import merge_charlotte as lib

%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
fname_business = '../data/yelp/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json'

In [3]:
with open(fname_business) as f:
    B = pd.DataFrame(json.loads(line) for line in f)

In [4]:
B.describe()

,latitude,longitude,open,review_count,stars
count,61184.000000,61184.000000,61184,61184.000000,61184.000000
mean,37.326077,-97.491803,0.878089,28.272506,3.673305
std,5.741085,29.624473,0.327186,88.652050,0.891207
min,32.871923,-115.386550,False,3.000000,1.000000
25%,33.509601,-115.052224,1,4.000000,3.000000
50%,35.994636,-111.932944,1,8.000000,3.500000
75%,36.240430,-80.863329,1,21.000000,4.500000
max,56.036545,8.549249,True,4578.000000,5.000000


In [5]:
B.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61184 entries, 0 to 61183
Data columns (total 15 columns):
attributes       61184 non-null object
business_id      61184 non-null object
categories       61184 non-null object
city             61184 non-null object
full_address     61184 non-null object
hours            61184 non-null object
latitude         61184 non-null float64
longitude        61184 non-null float64
name             61184 non-null object
neighborhoods    61184 non-null object
open             61184 non-null bool
review_count     61184 non-null int64
stars            61184 non-null float64
state            61184 non-null object
type             61184 non-null object
dtypes: bool(1), float64(3), int64(1), object(10)
memory usage: 7.1+ MB


In [6]:
B.head().T

,0,1,2,3,4
attributes,{u'By Appointment Only': True},"{u'Happy Hour': True, u'Accepts Credit Cards':...",{u'Good for Kids': True},{},"{u'Alcohol': u'full_bar', u'Noise Level': u'av..."
business_id,vcNAWiLM4dR7D2nwwJ7nCA,UsFtqoBl7naz8AVUBZMjQQ,cE27W9VPgO88Qxe4ol6y_g,HZdLhv6COCleJMo7nPl-RA,mVHrayjG3uZ_RLHkLj-AMg
categories,"[Doctors, Health & Medical]",[Nightlife],"[Active Life, Mini Golf, Golf]","[Shopping, Home Services, Internet Service Pro...","[Bars, American (New), Nightlife, Lounges, Res..."
city,Phoenix,Dravosburg,Bethel Park,Pittsburgh,Braddock
full_address,"4840 E Indian School Rd\nSte 101\nPhoenix, AZ ...","202 McClure St\nDravosburg, PA 15034","1530 Hamilton Rd\nBethel Park, PA 15234","301 S Hills Vlg\nPittsburgh, PA 15241","414 Hawkins Ave\nBraddock, PA 15104"
hours,"{u'Thursday': {u'close': u'17:00', u'open': u'...",{},{},"{u'Monday': {u'close': u'21:00', u'open': u'10...","{u'Tuesday': {u'close': u'19:00', u'open': u'1..."
latitude,33.4993,40.3505,40.3569,40.3576,40.4087
longitude,-111.984,-79.8869,-80.0159,-80.06,-79.8664
name,"Eric Goldberg, MD",Clancy's Pub,Cool Springs Golf Center,Verizon Wireless,Emil's Lounge
neighborhoods,[],[],[],[],[]


In [7]:
B.state.value_counts()

AZ     25230
NV     16485
NC      4963
QC      3921
PA      3041
EDH     2971
WI      2307
BW       934
IL       627
ON       351
SC       189
MLN      123
RP        13
ELN       10
FIF        4
CA         3
SCB        3
MN         1
MA         1
KHL        1
XGL        1
WA         1
NW         1
OR         1
NTH        1
HAM        1
Name: state, dtype: int64

In [8]:
B_keep = B[B.state.isin({'AZ','NV','PA','WI','IL','NC'})]

In [9]:
B_keep.groupby('state').describe()[['review_count', 'stars']].unstack(-1)

review_count                                               stars  \
             count       mean         std min 25% 50% 75%   max  count   
state                                                                    
AZ           25230  25.238962   55.871449   3   4   8  21  1512  25230   
IL             627  20.987241   33.924552   3   5   9  22   450    627   
NC            4963  20.651823   40.028150   3   4   8  19   983   4963   
NV           16485  45.672066  150.342727   3   5  10  29  4578  16485   
PA            3041  23.810917   43.332042   3   4   9  24   695   3041   
WI            2307  20.669267   37.064819   3   4   8  20   723   2307   

                                                 
           mean       std min 25%  50%  75% max  
state                                            
AZ     3.684503  0.940502   1   3  4.0  4.5   5  
IL     3.569378  0.897042   1   3  3.5  4.0   5  
NC     3.588858  0.844681   1   3  3.5  4.0   5  
NV     3.658477  0.908426   1   3  3.5  4.5   5  
PA     3.682999  0.829634   1   3  3.5  4.5   5  
WI     3.643693  0.857121   1   3  3.5  4.5   5

In [10]:
B_keep['type'].value_counts()

business    52653
Name: type, dtype: int64

# Merge with Health Inspections

In [4]:
def save_to_pickle(data, fname):
    with open(fname, 'wb') as handle:
        pickle.dump(data, handle)


def open_pickle(f_name):
    with open(f_name, 'rb') as f:
        data = pickle.load(f)
    return data

### Phoenix, AZ

In [626]:
test_AZ = open_pickle('../data/phx/phoenix_yelp_merge.pkl')

In [627]:
test_AZ.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3984 entries, (1130 the restaurant, 455, north 3rd st, phoenix, 85004) to (9990, north 90th st, scottsdale, 85258)
Data columns (total 43 columns):
address                    3984 non-null object
addressy_                  3984 non-null object
attributes                 3984 non-null object
avg_2                      2250 non-null float64
avg_w_2                    2250 non-null float64
business_id                3984 non-null object
categories                 3984 non-null object
city_                      3984 non-null object
complex                    3984 non-null object
complexy_                  3984 non-null object
cutting_edge               3984 non-null object
full_address               3984 non-null object
fuzz_partial_ratio         2250 non-null float64
fuzz_partial_ratio_2       2250 non-null float64
fuzz_ratio                 2250 non-null float64
fuzz_ratio_2               2250 non-null float64
fuzz_token_set_ratio       2250 n

### Charlotte, NC

In [402]:
def split_address(x):
    address = {'suite':''}
    x = x.lower().replace('\r','').replace('\n','')
    x = x.replace(' ste-',' ste ')
    
    words = x.split()
    # some street numbers have a dash, append the second part to suite
    if words[0].find('-') > 0:
        n = words[0].split('-')
        address['suite'] += n[-1] + ' '
        x = ' '.join([n[0]] + words[1:])
    
    x = x.replace('-',' ')
    x = re.sub('[%s]' % re.escape(string.punctuation.replace('&','')), '', x)
    abbr = {'road':'rd', 'street':'st', 'avenue':'av', 'ave':'av', 'drive':'dr', 'boulevard':'blvd',
            'lane':'ln', 'circle':'cir', 'building':'building', 'mount':'mt', 
            'n':'north', 'e':'east', 's':'south', 'w':'west', 'suite':'ste', 'bv':'blvd', 'suit':'ste',
            'pky':'pkwy', 'parkway':'pkwy',
            'first':'1st', 'second':'2nd', 'third':'3rd', 'fourth':'4th', 'fifth':'5th', 'sixth':'6th',
            'seventh':'7th', 'eighth':'8th', 'ninth':'9th', 'tenth':'10th'}
    for key, value in abbr.iteritems():
        x = re.sub(r'\b(%s)\b' % key, value, x) 
    x = x.strip()
    
    n = re.findall(r'^(\d+\D?)\b',x)
    s = re.search(r'\b(ste\W?\D?\W?\d*\W?\D?)|(unit\W?\D?\W?\d*\W?\D?)\Z', x)
    if len(n) > 0 and s is not None and len(s.group()) > 0:
        i = re.search(r'\b(ste)|(bldg)|(unit)\b', x)
        address['num'] = n[0]
        address['street'] = re.sub(r'^(\d+\D?)\b', '', x[:x.find(i.group())])
        address['suite'] += x[x.find(i.group()):]
    elif len(n) > 0:
        address['num'] = n[0]
        address['street'] = re.sub(r'^(\d+\D?)\b', '', x)
    else:
        address = {'num':'', 'street':x[0], 'suite':''}
        
    address = lib.strip_address_letters(address)
        
    # remove extra white spaces
    for key, value in address.iteritems():
        address[key] = re.sub(r'\s+', ' ', value).strip()
        
    return pd.Series(address)
    
def standard_name(x):
    rep_list = {'@':' at ', '-':' ', '/':' '}
    sub_list = {'grille':'grill', 'ristorante':'restaurant','restaurante':'restaurant',
                'italiano':'italian', 'mexicano':'mexican', 'mexicana':'mexican'}
    if type(x) == float:
        x = str(x)
    x = x.lower()
    for key, value in rep_list.iteritems():
        x = x.replace(key, value)
    for key, value in sub_list.iteritems():
        x = re.sub(r'\b(%s)\b' % key, value, x)
    
    x = x.strip()
    x = re.sub(r'(\#\d+)\Z','',x)
    x = re.split(r'\b(at)\b', x)[0].strip()
    return re.sub('[%s]' % re.escape(string.punctuation), '', x)

In [403]:
B_NC = B[(B.state=='NC')& (B.categories.apply(lambda x: 'Restaurants' in x))]
B_NC['address'] = B_NC.full_address.apply(lambda x: x.replace('\n',' ')\
                                                     .replace('Ste','Suite')\
                                                     .replace(',','')\
                                                     .replace(' NC ', ' '))

In [6]:
H = open_pickle('../data/char/char_FULL_04.pkl')

In [404]:
NC = pd.DataFrame.from_records(H).T
NC.head(1)

,address,city,county,inspections,name,state,type,zip
122131,1318 W MOREHEAD ST \r\n,CHARLOTTE,Mecklenburg,"[[2/26/2015, 90, A, 2398], [10/3/2014, 93, A, ...",OPEN KITCHEN,NC,1 - Restaurant,28208


In [405]:
NC['city_'] = NC.city.apply(lambda x: x.lower().strip())
NC['name_'] = NC.name.apply(standard_name)
NC['id_'] = NC.index

NC = pd.concat([NC, NC.address.apply(split_address)], axis=1)
NC[NC.id_==123786].head(1)

,address,city,county,inspections,name,state,type,zip,city_,name_,id_,num,street,suite
123786,516-A N GRAHAM ST \r\n,CHARLOTTE,Mecklenburg,"[[4/15/2015, 95, A, 2398], [11/19/2014, 91.50,...",SUB-ONE,NC,1 - Restaurant,28202,charlotte,sub one,123786,516,north graham st,a


In [406]:
B_address = pd.Series(zip(B_NC.full_address, B_NC.neighborhoods)).apply(lambda x: lib.parse_address(*x))
B_address.set_index(B_NC.index, inplace=True)
col = B_address.columns.values
col[0] = 'city_'
B_address.columns = col
B_NC = pd.concat([B_NC, B_address], axis=1)
B_NC['name_'] = B_NC.name.apply(standard_name)

In [242]:
B_NC[B_NC.business_id=='C3QfGqD3qjWUbwBOkEjqSQ'].full_address.tolist()

[u'9630 University City Blvd\nSte F\nUniversity City\nCharlotte, NC 28213']

In [407]:
NC_ind = ['name_','num','street','city_','zip']
temp = NC.set_index(NC_ind).join(B_NC.set_index(NC_ind), how='inner', rsuffix = 'y_')
temp.shape

(642, 27)

In [408]:
H_x = NC[~NC.id_.isin(temp.id_)]
B_x = B_NC[~B_NC.business_id.isin(temp.business_id)]

In [409]:
NC_ind2 = ['num','street','city_','zip']
temp2 = H_x.set_index(NC_ind2).join(B_x.set_index(NC_ind2), how='inner', rsuffix = 'y_')

In [410]:
temp2.shape

(1598, 29)

In [ ]:
temp2.to_csv('../data/char/merge_dump_01.csv', encoding='utf-8')

In [411]:
temp_fuzz = lib.append_fuzz_scores(temp2)

In [412]:
temp_fuzz.to_csv('../data/char/merge_dump_07.csv', encoding='utf-8')

In [216]:
reload(lib)

<module 'merge_charlotte' from 'merge_charlotte.py'>

In [413]:
ind_A = (temp_fuzz['max'] >= 75) & (temp_fuzz.avg_w_2 >= 75)
ind_B = (temp_fuzz['max'] >= 60) & (temp_fuzz.avg_w_2 >= 60) & (temp_fuzz.avg_w_3 >= 80)
NC_level2 = pd.concat([temp, temp_fuzz[ind_A | ind_B]])

In [414]:
NC_level2.shape

(1134, 52)

In [415]:
H_x2 = NC[~NC.id_.isin(NC_level2.id_)]
B_x2 = B_NC[~B_NC.business_id.isin(NC_level2.business_id)]

In [416]:
NC_ind3 = ['city_','zip']
temp3 = H_x2.set_index(NC_ind3).join(B_x2.set_index(NC_ind3), how='inner', rsuffix = 'y_')

In [417]:
temp3.shape

(54046, 33)

In [418]:
temp_fuzz3 = lib.append_fuzz_scores(temp3)

In [419]:
ind_A3 = (temp_fuzz3['max'] >= 75) & (temp_fuzz3.avg_w_2 >= 75)
ind_B3 = (temp_fuzz3['max'] >= 60) & (temp_fuzz3.avg_w_2 >= 60) & (temp_fuzz3.avg_w_3 >= 80)
temp_fuzz3[ind_A3 | ind_B3].to_csv('../data/char/merge_dump_12.csv', encoding='utf-8')

In [440]:
def to_int(x):
    try:
        out = int(x)
    except:
        out = np.nan
    return out

ind_A3 = (temp_fuzz3['max'] >= 75) & (temp_fuzz3.avg_w_2 >= 75)
ind_B3 = (temp_fuzz3['max'] >= 60) & (temp_fuzz3.avg_w_2 >= 60) & (temp_fuzz3.avg_w_3 >= 80)
ind_C3 = temp_fuzz3.num == temp_fuzz3.numy_
ind_D3 = (temp_fuzz3.street == temp_fuzz3.street) & \
         (abs(temp_fuzz3.num.apply(to_int) - temp_fuzz3.numy_.apply(to_int)) <= 100)
NC_level3 = pd.concat([NC_level2, temp_fuzz3[(ind_A3 | ind_B3) & (ind_C3 | ind_D3)]])

In [442]:
NC_level3.shape

(1332, 56)

In [443]:
H_x3 = NC[~NC.id_.isin(NC_level3.id_)]
B_x3 = B_NC[~B_NC.business_id.isin(NC_level3.business_id)]

In [445]:
save_to_pickle(NC_level3,'../data/char/charlotte_yelp_merge.pkl')

In [444]:
print H_x3.shape
print B_x3.shape

(1602, 14)
(764, 23)


In [284]:
reload(lib)

<module 'merge_charlotte' from 'merge_charlotte.py'>

In [430]:
b_list = [129641,129644,129560,151433,127252,128514,128700,124251,128677,130023,123415,
          126428,126920,160381,123786,124262,127957,144402]
bad_list = ['lmY05KHoqyFZjGsuHYx-HQ', 'SNqqdZNsC1w8mL15SHJEWw', 
            'WjVUvwDBosUunBm3uY-pyA', 'cE8IX3c8V47cBou08KsklA',
            'T1jYZFB_7cqdhuvzpxfFWQ', 'yvNnNl1DqVRmVuvCBytxuw',
            '4jl0_Ny3wForCqs06DRzRg', 'tZ1FVdbAna3lUL5LGcAa_Q',
            'cEtlUnEBXDPsFHhhou27tw', 'RguKChB4sdPwYA_1_S6JMg',
            'HKwlJatD3lZjzmF8n_9dNA', 'mH3fUPIdmh5ybrmD48pJkQ',
            '6ZtftjQ18WUN3-IW_yA3tQ', 'Iu0ElolxFlhCKimijFTPNw',
            'h2fqHWVr6jmjqpwr3gELzg', 'JF4Ge_KqznXoZ-zmk_veQw']
bad_ind = B_NC.business_id.isin(bad_list)
b_ind = NC.id_.isin(b_list)

In [431]:
pd.Series(zip(B_NC[bad_ind].full_address, B_NC[bad_ind].neighborhoods)).apply(lambda x: lib.parse_address(*x))

,city,complex,num,street,suite,zip
0,pineville,,9940,pineville matthews rd,,28134
1,matthews,,1905,matthews township pkwy,,28105
2,charlotte,,10223,university city blvd,,28213
3,charlotte,,1601,elizabeth av,,28204
4,charlotte,,8542,university city blvd,,28213
5,charlotte,charlotte douglas international airport,5501,r c josh birmingham pkwy,,28208
6,charlotte,,1100,metropolitan av,bldg east ste 100,28204
7,charlotte,,8332,pinevillematthews rd,ste 201203,28226
8,charlotte,,10405,park rd,b,28210
9,pineville,,9101,pineville matthews rd,,28134


In [432]:
B_NC[bad_ind][['full_address']]

,full_address
720,"9940 Pineville Matthews Rd\nPineville, NC 28134"
9214,"1905 Matthews Township Parkway\nMatthews, NC 2..."
9621,10223 University City Blvd\nUniversity City\nC...
22748,"1601 Elizabeth Ave\nElizabeth\nCharlotte, NC 2..."
26452,8542 University City Blvd\nUniversity City\nCh...
31718,Charlotte Douglas International Airport\n5501 ...
31798,"1100 Metropolitan Ave\nBldg E, Ste 100\nMyers ..."
31984,8332 Pineville-Matthews Rd\nSte 201-203\nCharl...
32788,"10405-B Park Rd\nCharlotte, NC 28210"
33087,"9101 Pineville Matthews Rd\nPineville, NC 28134"


In [428]:
NC[b_ind].address.apply(split_address)

,num,street,suite
123415,1600,matthews mint hill rd,
123786,516,north graham st,a
124251,4520,north tryon st,41
124262,710,west trade st,g
126428,2910,mt holly huntersville rd,
126920,9005,j m keynes blvd,2
127252,440,east mccullough dr,230
127957,265,mtholly huntersville rd,
128514,10405,park rd,
128677,15105,john j delaney dr,a


In [429]:
NC[b_ind][['address']]

,address
123415,1600 MATTHEWS-MINT HILL RD \r\n
123786,516-A N GRAHAM ST \r\n
124251,4520-41 N TRYON ST \r\n
124262,710-G W TRADE ST \r\n
126428,2910 MT. HOLLY-HUNTERSVILLE RD \r\n
126920,9005-2 J M KEYNES BLVD \r\n
127252,440-230 E MCCULLOUGH DR \r\n
127957,265 MT.HOLLY-HUNTERSVILLE RD \r\n
128514,10405 PARK RD \r\n
128677,15105-A JOHN J DELANEY DR \r\n


In [375]:
y = u'1100 Metropolitan Ave Bldg E Ste 100 Myers'.lower()
x = re.search(r'\b(ste)|(bldg)|(unit)\b',y)

In [322]:
re.split(r'\b%s\b' % 'Pineville', '9101 Pineville Matthews Rd\nPineville, NC 28134')

['9101 ', ' Matthews Rd\n', ', NC 28134']

In [315]:
y[y.find(x.group()):]

u'bldg e ste 100 myers'

In [329]:
x.group()

u'bldg'